In [3]:
#RUN THAT
library(gsheet)
library(dplyr)
library(tidyr)

In [168]:
#YleTopicIdCity  <- readRDS("YleTopicIdCity1")

In [2]:
#RUN THAT
YleTopicIdCity  <- readRDS("YleTopicIdCity")
intFedTopicIdCity <- readRDS("intFedTopicIdCity")
intRegTopicIdCity  <- readRDS("intRegTopicIdCity ")
yle <- read.csv("data/processed/yle_fi_lemmas.csv",stringsAsFactor = F)
integrum <- read.csv("data/processed/integrum_federal.csv",stringsAsFactor = F)
integrumLocal <- read.csv("data/processed/integrum_local.csv",stringsAsFactor = F)

In [160]:
YleTopicIdCity  <- readRDS("YleTopicIdCity1")

In [89]:
IntegrumFiCities <- readRDS("IntegrumFiCities")

In [174]:
#NUMERATE TOPICS HERE
#TopicNums <- c(3,12,20) #fedral culture
#TopicNums <- c(4,5,8,12,18)# local culture 
#TopicNums <- c(1,5,8,9,15,28,30,19,22) #fedral economy
#TopicNums <- c(1,2,3,6,9,11,16,24,26,29) #local economy

TopicNums <- c( 6,14,24,28) #yle economy
#TopicNums <- c( 4,30) #yle culture


#Name the topic
TopicName <- "Economy"
# Pick one "yle","IntegrumREG","IntegrumFED"
CorpusName <- "yle"


#Can also
#Select periods (upper is included) (1996,2001] - 2001 is included 1996 nottem
Periods <-  c(1990,1996,2001,2006,2011,2017)
TopicNames <- paste(rep("Topic",length(TopicNums)),TopicNums,sep="")

In [170]:
#GivenTopicCities%>% head(50)
YleTopicIdCity%>%  str


'data.frame':	1372770 obs. of  7 variables:
 $ id             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Topic          : chr  "Topic1" "Topic1" "Topic1" "Topic1" ...
 $ TopicProportion: chr  "6.4256634169303e-06" "6.4256634169303e-06" "6.4256634169303e-06" "6.4256634169303e-06" ...
 $ tokensLong     : chr  "Vladimir" "Vladimir" "Moskova" "Moskova" ...
 $ Fi             : chr  "Vladimir" "Vladimir" "Moskova" "Moskova" ...
 $ N              : num  1 1 1 1 1 1 1 1 1 1 ...
 $ Ru             : chr  "Владимир" "Владимир" "Москва" "Москва" ...


In [155]:
TopicNames

[1] "Topic4"  "Topic30"

In [175]:
if(CorpusName == "IntegrumREG")
    {
     GivenTopicCities <-intRegTopicIdCity %>% filter(Topic %in% TopicNames) %>% 
    filter(TopicProportion > 0.3)   %>% mutate(city = tokensLong) %>% select(-tokensLong)
    GivenTopicCities <-  GivenTopicCities %>% left_join(IntegrumFiCities %>% mutate(Ru = tolower(Ru)), by = c("city" = "Ru")) 
    GivenTopicCities <-  GivenTopicCities %>% left_join(integrumLocal  %>% select(id,year)) 
      GivenTopicCities <-  GivenTopicCities %>% mutate(city=Fi) 
    print(paste(TopicName,CorpusName))
    }
if(CorpusName == "IntegrumFED")
    {
     GivenTopicCities <-intFedTopicIdCity %>% filter(Topic %in% TopicNames) %>% 
    filter(TopicProportion > 0.3)   %>% mutate(city = tokensLong) %>% select(-tokensLong) 
    GivenTopicCities <-  GivenTopicCities %>% left_join(IntegrumFiCities %>% mutate(Ru = tolower(Ru)), by = c("city" = "Ru")) 
    GivenTopicCities <-  GivenTopicCities %>% left_join(integrum  %>% select(id,year))
      GivenTopicCities <-  GivenTopicCities %>% mutate(city=Fi)
    print(paste(TopicName,CorpusName))
     }
if(CorpusName == "yle")
    {
     GivenTopicCities <- YleTopicIdCity %>% filter(Topic %in% TopicNames) %>% 
    filter(TopicProportion > 0.3)   %>% mutate(city = Ru) %>% select(-tokensLong) 
    GivenTopicCities <-  GivenTopicCities %>% left_join(yle %>% mutate(id = X)  %>% select(id,year)) 
     print(paste(TopicName,CorpusName))
    }

Joining, by = "id"


[1] "Economy yle"


In [162]:
YleTopicIdCity %>% str

'data.frame':	1372770 obs. of  6 variables:
 $ id             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Topic          : chr  "Topic1" "Topic1" "Topic1" "Topic1" ...
 $ TopicProportion: chr  "6.4256634169303e-06" "6.4256634169303e-06" "6.4256634169303e-06" "6.4256634169303e-06" ...
 $ tokensLong     : chr  "Vladimir" "Vladimir" "Moskova" "Moskova" ...
 $ Fi             : chr  "Vladimir" "Vladimir" "Moskova" "Moskova" ...
 $ N              : num  1 1 1 1 1 1 1 1 1 1 ...


In [176]:
GivenTopicCities <- GivenTopicCities %>% mutate(periods = cut(year,breaks = Periods)) 
CityPeriodFreq <- GivenTopicCities %>% group_by(city,periods) %>% summarise(CityFreq = n_distinct(id)) %>% 
arrange(city,periods)
FiveYperiods <- sort(unique(CityPeriodFreq$periods))
dir1 <- paste("Places/",TopicName,CorpusName,Sys.time(),sep="")
dir.create(dir1)

In [177]:
for(i in 1:length(FiveYperiods))
    {
    filenam <-  file.path(dir1,paste(TopicName,CorpusName,FiveYperiods[i],".csv",sep=""))
    write.csv(CityPeriodFreq %>% filter(periods==FiveYperiods[i]),file = filenam, row.names = F,
          col.names = F)
    }

Warning message in write.csv(CityPeriodFreq %>% filter(periods == FiveYperiods[i]), :
“attempt to set 'col.names' ignored”Warning message in write.csv(CityPeriodFreq %>% filter(periods == FiveYperiods[i]), :
“attempt to set 'col.names' ignored”Warning message in write.csv(CityPeriodFreq %>% filter(periods == FiveYperiods[i]), :
“attempt to set 'col.names' ignored”Warning message in write.csv(CityPeriodFreq %>% filter(periods == FiveYperiods[i]), :
“attempt to set 'col.names' ignored”